In [ ]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

## Open the database and set up for running the algorithm 

In [ ]:
path_to_db_folder = '' #string path to folder where database is
ds = pd.read_excel('{0}\\Globe-LFMC-2.0.xlsx'.format(path_to_db_folder), sheet_name='LFMC data')

In [ ]:
globelfmc = ds.copy()
del ds
globelfmc = globelfmc.sort_values(by=['Sorting ID'])
globelfmc = globelfmc.dropna(subset=['LFMC value (%)']) # in case there are empty rows in the spreadsheet

In [ ]:
# transform dates in timestamps to use as input in the statistical model
globelfmc['timestamp'] = pd.to_datetime(globelfmc['Sampling date (YYYYMMDD)'].values).map(lambda x: pd.Timestamp(x).timestamp())

In [ ]:
# in case there are spaces at beginning or end
globelfmc['Species collected nospace'] = globelfmc['Species collected'].str.strip()
globelfmc['Site name nospace'] = globelfmc['Site name'].str.strip()

In [ ]:
# columns with Cook's Distance

globelfmc['Above 4/n Cook Distance'] = np.nan
globelfmc['Above 3xMean Cook Distance'] = np.nan
globelfmc['Cook Distance'] = np.nan
globelfmc['Mean Cook Distance'] = np.nan
globelfmc['3x mean Cook Distance'] = np.nan
globelfmc['n for Cook Distance'] = np.nan
globelfmc['4/n for Cook Distance'] = np.nan

## Run algorithm and fill columns

In [ ]:
sites = sorted(set(globelfmc['Site name nospace']))
len_sites = len(sites)

for i,site in enumerate(sites):
    print(i+1, len_sites)
    species_list = sorted(set(globelfmc.loc[globelfmc['Site name nospace'] ==  site, 'Species collected nospace']))

    for species in species_list:
        
        Y = globelfmc.loc[(globelfmc['Site name nospace']==site) & (globelfmc['Species collected nospace']==species), 'LFMC value (%)']
        X = globelfmc.loc[(globelfmc['Site name nospace']==site) & (globelfmc['Species collected nospace']==species), 'timestamp']
        X = sm.add_constant(X) 

        n = len(X)

        globelfmc.loc[(globelfmc['Site name nospace']==site) & (globelfmc['Species collected nospace']==species), 'n for Cook Distance'] = n
        globelfmc.loc[(globelfmc['Site name nospace']==site) & (globelfmc['Species collected nospace']==species), '4/n for Cook Distance'] = 4/n

        ols = sm.OLS(Y,X).fit()
        cooks_d = ols.get_influence().cooks_distance

        globelfmc.loc[(globelfmc['Site name nospace']==site) & (globelfmc['Species collected nospace']==species), 'Cook Distance'] = cooks_d[0] #first array is actual cook's distance, second array is p values
        globelfmc.loc[(globelfmc['Site name nospace']==site) & (globelfmc['Species collected nospace']==species), 'Mean Cook Distance'] = np.mean(cooks_d[0])
        globelfmc.loc[(globelfmc['Site name nospace']==site) & (globelfmc['Species collected nospace']==species), '3x mean Cook Distance'] = 3 * np.mean(cooks_d[0])

        out_bool_4n = cooks_d[0] > globelfmc.loc[(globelfmc['Site name nospace']==site) & (globelfmc['Species collected nospace']==species), '4/n for Cook Distance']
        globelfmc.loc[(globelfmc['Site name nospace']==site) & (globelfmc['Species collected nospace']==species), 'Above 4/n Cook Distance'] = out_bool_4n

        out_bool_3m = cooks_d[0] > globelfmc.loc[(globelfmc['Site name nospace']==site) & (globelfmc['Species collected nospace']==species), '3x mean Cook Distance']
        globelfmc.loc[(globelfmc['Site name nospace']==site) & (globelfmc['Species collected nospace']==species), 'Above 3xMean Cook Distance'] = out_bool_3m




In [ ]:
# if cook's distance infinite or nan, write NA

condition = (globelfmc['Cook Distance']==np.inf) | (pd.isna(globelfmc['Cook Distance'])) | (globelfmc['Mean Cook Distance']==np.inf) | (pd.isna(globelfmc['Mean Cook Distance']))

globelfmc.loc[condition, ['Cook Distance','Mean Cook Distance','3x mean Cook Distance','Above 4/n Cook Distance','Above 3xMean Cook Distance']] = 'NA' 



## Save output

In [ ]:
globelfmc.to_excel('{}\\Globe-LFMC-2.0_outliers_CD.xlsx'.format(path_to_db_folder), index=False)